Importing the required libraries for our project

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
import seaborn as sns

import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image

import tensorflow as tf
from tensorflow.keras.applications import (VGG16, MobileNetV2, NASNetMobile, ResNet50)


training and testing dataset paths 

In [ ]:
TRAIN_PATH = "/content/drive/MyDrive/computer vision/Ultra-sound covid detection/Train"
VAL_PATH = "/content/drive/MyDrive/computer vision/Ultra-sound covid detection/Test"

# Custom neural network with VGG 16 Model on top

Building the VGG-16 network on top of our custom model and using the convolutional part of the network 

In [ ]:
# declared the VGG 16 network by creating an object named vgg16model
vgg16model = VGG16()

In [ ]:
# Mentioned the different layers present within this network using summary()
vgg16model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
print(type(vgg16model))

vggconv_model = Sequential() # sequential models are linear stacked layers where each layer has exactly one input tensor and one output tensor

# removed the last layers of the model and add custom layers to it by converting the functional model into sequential model and removing the last layer of 1000 classifications
for layer in vgg16model.layers[:-1]:
  vggconv_model.add(layer)

print(type(vggconv_model))

<class 'tensorflow.python.keras.engine.functional.Functional'>
<class 'tensorflow.python.keras.engine.sequential.Sequential'>


In [ ]:
# mentioned the newly generated model using the summary()
vggconv_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [ ]:
# freeze the layers since Freezing a layer prevents its weights from being modified. Since for transfer learning we need not to train the upper part so it is used up as non-trainable parameters
for layer in vggconv_model.layers:

  layer.trainable = False

In [ ]:
# adding custom layers 
vggconv_model.add(Dense(64,activation='relu'))
vggconv_model.add(Dropout(0.5))
vggconv_model.add(BatchNormalization())
vggconv_model.add(Dense(1,activation='sigmoid'))


vggconv_model.compile(loss= keras.losses.binary_crossentropy,optimizer='adam',metrics=['accuracy'])

In [ ]:
vggconv_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

Using the data augmentation techniques to reduce the over-fitting of the model

In [ ]:
# train from scratch by creating image data generator alongwith data augmentation techniques to reduce the data overfitting

train_datagen = image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 10,
    horizontal_flip = True,
    vertical_flip = True
)

# for test dataset only rescaling the images and don't need variability
test_datagen = image.ImageDataGenerator(rescale = 1./255)

In [ ]:
# we mentioned the training path of our data ceated the data generator and its advantage.
train_generator = train_datagen.flow_from_directory( 
    TRAIN_PATH,
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary'
)

Found 519 images belonging to 2 classes.


In [ ]:
# for testing dataset we created the validation generator with the same approach
validation_generator = test_datagen.flow_from_directory(
    VAL_PATH,
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary'
)

Found 191 images belonging to 2 classes.


In [ ]:
# mentioned the classes of both the dataset
print(train_generator.class_indices)
print(validation_generator.class_indices)

{'Covid': 0, 'Normal': 1}
{'Covid': 0, 'Normal': 1}


Training our model with following conditions:
steps_epoch as 6, epocs size 20 with 2 validations for each steps 

In [ ]:
hist = vggconv_model.fit(
    train_generator,
    steps_per_epoch=6,
    epochs = 20,
    validation_data = validation_generator,
    validation_steps=2
)

Epoch 1/20
6/6 [==============================] - 258s 43s/step - loss: 0.7508 - accuracy: 0.5052 - val_loss: 0.4508 - val_accuracy: 0.9062
Epoch 2/20
6/6 [==============================] - 122s 20s/step - loss: 0.5752 - accuracy: 0.7083 - val_loss: 0.3865 - val_accuracy: 0.8906
Epoch 3/20
6/6 [==============================] - 66s 11s/step - loss: 0.5642 - accuracy: 0.7344 - val_loss: 0.4908 - val_accuracy: 0.7344
Epoch 4/20
6/6 [==============================] - 46s 8s/step - loss: 0.5003 - accuracy: 0.7844 - val_loss: 0.3623 - val_accuracy: 0.9062
Epoch 5/20
6/6 [==============================] - 24s 4s/step - loss: 0.6042 - accuracy: 0.6287 - val_loss: 0.4259 - val_accuracy: 0.7656
Epoch 6/20
6/6 [==============================] - 15s 2s/step - loss: 0.5855 - accuracy: 0.7083 - val_loss: 0.4540 - val_accuracy: 0.7812
Epoch 7/20
6/6 [==============================] - 10s 2s/step - loss: 0.5159 - accuracy: 0.7552 - val_loss: 0.3253 - val_accuracy: 0.9844
Epoch 8/20
6/6 [=============

In [ ]:
vggconv_model.evaluate(train_generator)

17/17 [==============================] - 9s 530ms/step - loss: 0.4439 - accuracy: 0.8979


[0.4438990652561188, 0.8978805541992188]

In [ ]:
vggconv_model.evaluate(validation_generator)

#yhat_probs = model.predict(validation_generator, verbose=0)
#yhat_classes = model.predict_classes(validation_generator, verbose=0)

6/6 [==============================] - 4s 714ms/step - loss: 0.3964 - accuracy: 0.9529


[0.3964141011238098, 0.9528796076774597]

In [ ]:
vggconv_model.save("vggconv_model_adv1.h5")

In [ ]:
model = load_model("vggconv_model_adv1.h5")

# Custom neural network with MobileNet V2 Model on top

In [ ]:
input_size: tuple = (224, 224, 3)
hidden_size: int = 64
dropout: float = 0.5
num_classes: int = 3
trainable_layers: int = 0
log_softmax: bool = False

# freezing out the non-trainable layers with the help of this function
def fix_layers(model, num_flex_layers: int = 1):
    """
    Receives a model and freezes all layers but the last num_flex_layers ones.
    Arguments:
        model [tensorflow.python.keras.engine.training.Model] -- model
    Keyword Arguments:
        num_flex_layers {int} -- [Number of trainable layers] (default: {1})
    Returns:
        Model -- updated model
    """
    num_layers = len(model.layers)
    for ind, layer in enumerate(model.layers):
        if ind < num_layers - num_flex_layers:
            layer.trainable = False

    return model

act_fn = tf.nn.softmax if not log_softmax else tf.nn.log_softmax    

In [ ]:
# using the weights of the mobilenet v2 model which was pretrained on imagenet dataset, using include_top=false we are discarding the last layer from our model
# also the concept of converting into sequential networkis shown by the third parameter

mobilenetmodel = MobileNetV2(weights="imagenet",include_top=False,input_tensor=Input(shape=input_size))

9412608/9406464 [==============================] - 0s 0us/step


In [ ]:
mobilenetmodel.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

In [ ]:
headModel = mobilenetmodel.output
headModel = AveragePooling2D(pool_size=(4, 4))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(hidden_size)(headModel)
headModel = BatchNormalization()(headModel)
headModel = ReLU()(headModel)
headModel = Dropout(dropout)(headModel)
headModel = Dense(num_classes, activation=act_fn)(headModel)

model_mobilenet = Model(inputs=mobilenetmodel.input, outputs=headModel)
model_mobilenet = fix_layers(model_mobilenet, num_flex_layers=trainable_layers + 8)

In [ ]:
model_mobilenet.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________________

In [ ]:
model_mobilenet.compile(loss= keras.losses.binary_crossentropy,optimizer='adam',metrics=['accuracy'])

In [ ]:
# train from scratch

train_datagen = image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 10,
    horizontal_flip = True,
    vertical_flip = True
)

test_datagen = image.ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory( 
    TRAIN_PATH,
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary'
)

Found 519 images belonging to 2 classes.


In [ ]:
validation_generator = test_datagen.flow_from_directory(
    VAL_PATH,
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary'
)

Found 191 images belonging to 2 classes.


In [ ]:
print(train_generator.class_indices)
print(validation_generator.class_indices)

{'Covid': 0, 'Normal': 1}
{'Covid': 0, 'Normal': 1}


In [ ]:
hist = model_mobilenet.fit(
    train_generator,
    steps_per_epoch=6,
    epochs = 18,
    validation_data = validation_generator,
    validation_steps=2
)

Epoch 1/18
6/6 [==============================] - 5s 751ms/step - loss: 1.1354 - accuracy: 0.3750 - val_loss: 0.9732 - val_accuracy: 0.1250
Epoch 2/18
6/6 [==============================] - 3s 557ms/step - loss: 1.0521 - accuracy: 0.3333 - val_loss: 1.0010 - val_accuracy: 0.2812
Epoch 3/18
6/6 [==============================] - 3s 535ms/step - loss: 1.0710 - accuracy: 0.3174 - val_loss: 1.0289 - val_accuracy: 0.4531
Epoch 4/18
6/6 [==============================] - 4s 601ms/step - loss: 0.9700 - accuracy: 0.3073 - val_loss: 1.0532 - val_accuracy: 0.5625
Epoch 5/18
6/6 [==============================] - 4s 609ms/step - loss: 0.9606 - accuracy: 0.3054 - val_loss: 0.8695 - val_accuracy: 0.3750
Epoch 6/18
6/6 [==============================] - 4s 602ms/step - loss: 0.9479 - accuracy: 0.2969 - val_loss: 0.8379 - val_accuracy: 0.5312
Epoch 7/18
6/6 [==============================] - 3s 579ms/step - loss: 0.9379 - accuracy: 0.3281 - val_loss: 0.8459 - val_accuracy: 0.4531
Epoch 8/18
6/6 [====

In [ ]:
print(model_mobilenet.evaluate(train_generator))
print(model_mobilenet.evaluate(validation_generator))

17/17 [==============================] - 8s 483ms/step - loss: 0.8184 - accuracy: 0.1869
[0.8184353113174438, 0.186897873878479]
6/6 [==============================] - 1s 216ms/step - loss: 0.8403 - accuracy: 0.1257
[0.8403012752532959, 0.1256544440984726]


In [ ]:
model_mobilenet.save("model_mobilenet_adv1.h5")

# Custom neural network with NasNet mobile Model on top

In [ ]:
nasnetModel = NASNetMobile(
        weights="imagenet",
        include_top=False,
        input_tensor=Input(shape=input_size)
    )
    # construct the head of the model that will be placed on top of the
    # the base model
headModel = nasnetModel.output
headModel = AveragePooling2D(pool_size=(4, 4))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(hidden_size)(headModel)
headModel = ReLU()(headModel)
headModel = Dropout(dropout)(headModel)
headModel = BatchNormalization()(headModel)
headModel = Dense(num_classes, activation=act_fn)(headModel)

# place the head FC model on top of the base model
model_nasnet = Model(inputs = nasnetModel.input, outputs=headModel)

model_nasnet = fix_layers(model_nasnet, num_flex_layers=trainable_layers + 8)

19996672/19993432 [==============================] - 0s 0us/step


In [ ]:
model_nasnet.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 111, 111, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 111, 111, 32) 128         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation_188 (Activation)     (None, 111, 111, 32) 0           stem_bn1[0][0]                   
_______________________________________________________________________________________

In [ ]:

model_nasnet.compile(loss= keras.losses.binary_crossentropy,optimizer='adam',metrics=['accuracy'])

In [ ]:
# train from scratch

train_datagen = image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 10,
    horizontal_flip = True,
    vertical_flip = True
)

test_datagen = image.ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory( 
    TRAIN_PATH,
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary'
)

Found 519 images belonging to 2 classes.


In [ ]:
validation_generator = test_datagen.flow_from_directory(
    VAL_PATH,
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary'
)

Found 191 images belonging to 2 classes.


In [ ]:
print(train_generator.class_indices)
print(validation_generator.class_indices)

{'Covid': 0, 'Normal': 1}
{'Covid': 0, 'Normal': 1}


In [ ]:
hist = model_nasnet.fit(
    train_generator,
    steps_per_epoch=6,
    epochs = 18,
    validation_data = validation_generator,
    validation_steps=2
)

Epoch 1/18
6/6 [==============================] - 53s 9s/step - loss: 1.0555 - accuracy: 0.3438 - val_loss: 0.9519 - val_accuracy: 0.3281
Epoch 2/18
6/6 [==============================] - 52s 9s/step - loss: 0.9610 - accuracy: 0.3653 - val_loss: 0.9386 - val_accuracy: 0.2656
Epoch 3/18
6/6 [==============================] - 28s 5s/step - loss: 0.9308 - accuracy: 0.3293 - val_loss: 0.9283 - val_accuracy: 0.3438
Epoch 4/18
6/6 [==============================] - 19s 3s/step - loss: 0.9515 - accuracy: 0.4115 - val_loss: 0.9330 - val_accuracy: 0.5781
Epoch 5/18
6/6 [==============================] - 15s 2s/step - loss: 0.9278 - accuracy: 0.3952 - val_loss: 0.8932 - val_accuracy: 0.2188
Epoch 6/18
6/6 [==============================] - 7s 1s/step - loss: 0.9259 - accuracy: 0.3125 - val_loss: 0.8486 - val_accuracy: 0.2500
Epoch 7/18
6/6 [==============================] - 4s 681ms/step - loss: 0.8751 - accuracy: 0.3353 - val_loss: 0.8712 - val_accuracy: 0.1250
Epoch 8/18
6/6 [=================

In [ ]:
print(model_nasnet.evaluate(train_generator))
print(model_nasnet.evaluate(validation_generator))

17/17 [==============================] - 8s 468ms/step - loss: 0.7977 - accuracy: 0.5376
[0.7977374196052551, 0.5375722646713257]
6/6 [==============================] - 1s 205ms/step - loss: 0.8281 - accuracy: 0.6387
[0.8280627727508545, 0.6387434601783752]


# Deployment of VGG-16 based model on web using Streamlit

In [ ]:
# saving the vgg-16 model in hdf5 format

tf.keras.models.save_model(model,"my_model.hdf5")


In [ ]:
# for creating datascience web apps
!pip install streamlit

In [ ]:
!pip install pyngrok

In [ ]:
# whatever we write here will be saved as a different python file in app.py
%%writefile app.py  
import streamlit as st
import tensorflow as tf

st.set_option("deprecation.showfileUploaderEncoding",False) # whenever installing streamlit app it shows warning to avoid it we put it false

# when we load our big model and if there exist some changes it reloads it which is time consuming so function wriiten below will be run once and stored in cache memory
@st.cache(allow_output_mutation = True) 

def load_model():
  model = tf.keras.models.load_model("/content/drive/MyDrive/computer vision/Ultra-sound covid detection/my_model.hdf5")
  return model

# loading our model
model = load_model  

# giving title toour webpage
st.write("""
        # COVID-19 detection
        """
         )
# uploading the image in web
file = st.file_uploader("Please upload an Ultrasound chest image", type = ["jpg","png"])
import cv2
from PIL import Image,ImageOps
import numpy as np

# a function which accepts image and the model and results prediction
def import_and_predict(image_data,model):

  # resize the image to the dimension needed for our model to work
  size = (180,180)
  image = ImageOps.fit(image_data,size,Image.ANTIALIAS)
  img = np.asarray(image)

  # we have 3 dim. i.e. lx b x color-channels plus 4th dim. is no. of images so we create a new dim
  img_reshape = img[np.newaxis,...]
  prediction = model.predict(img_reshape)

  return prediction

# if no file is uploaded then print msg
if file is None:
  st.text("please upload an image file")

else:

  # if user uploaded img then open img
  image = Image.open(file)

  # displaying the image
  st.image(image,use_column_width=True)

  # calling the func to import and predict its class
  predictions = import_and_predict(image,model)
  class_names = ["Covid","Normal"]
  string = "Image most likely is:"+class_names[np.argmax(predictions)]
  
  # print the string in a box
  st.success(string)



Writing app.py


In [ ]:
!mkdir -p /drive/ngrok-ssh
%cd /drive/ngrok-ssh
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip
!cp /drive/ngrok-ssh/ngrok /ngrok
!chmod +x /ngrok

/drive/ngrok-ssh
--2020-11-29 16:25:26--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.196.106.64, 54.164.152.149, 3.226.231.47, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.196.106.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  33.6MB/s    in 0.4s    

2020-11-29 16:25:26 (33.6 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
!/ngrok authtoken 1kpA8XicQ05NHoHG2B1CxzLNqRJ_6VXpV7qnXhkyXMfJ63E2p

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
#get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')

In [ ]:
!nohup streamlit run app.py &

nohup: appending output to 'nohup.out'


In [ ]:
from pyngrok import ngrok

url = ngrok.connect(port=8501)
url

<NgrokTunnel: "http://e8a1434241ea.ngrok.io" -> "http://localhost:80">